In [ ]:
import numpy as np
import pandas as pd
from sklearn.manifold import MDS
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from MainExample import DeepMDA
import snf
# matplotlib.use('agg')
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
import matplotlib
# from keras.layers.core import Dropout, Activation
from tensorflow.python.keras.layers import Dense, Dropout, Activation, InputLayer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.python.keras.utils import np_utils

In [1]:
# Generate list of drug names, like : ['drug1', 'drug2', ....] 
def get_drug_name_list():
    drug_name_list = list()
    for index in range(1, 549):
        drug_name_list.append("drug" + str(index))
    return drug_name_list

In [ ]:
# Read similarity matrix file and return as numpy array
def read_similarity_matrix(path: str):
    #drug_name_list = get_drug_name_list()
    df = pd.read_csv(path, index_col=False, header=None, dtype=np.float64)
    return df.to_numpy()

In [ ]:
# Finds the candidate drugs of clustering
def find_candidate(_embeddings, _kmeans):
    centroids = _kmeans.cluster_centers_
    closest, _ = pairwise_distances_argmin_min(centroids, _embeddings)

    # Central samples of each cluster
    central_samples = _embeddings[closest]

    indices = np.zeros(len(central_samples))
    for i in range(len(central_samples)):
        tmp, _ = np.where(np.isclose(central_samples[i], _embeddings))
        indices[i] = tmp[0]

    return list(indices.astype(int))

In [ ]:
# Does the clustering or grouping of dataset
def make_groups(_embeddings, group_count):
    _kmeans = KMeans(n_clusters=group_count)  # Adjust the number of clusters as needed
    return _kmeans.fit_predict(_embeddings), _kmeans

In [ ]:
# Prepare the input data for Neural network
def prepare_data(candidates=None, seperate=False):
    # Uncomment for DS1
    drug_fea = np.loadtxt("NDD/DS1/IntegratedDS1.txt",dtype=float,delimiter=",")
    interaction = np.loadtxt("NDD/DS1/drug_drug_matrix.csv",dtype=int,delimiter=",")
    # Uncomment for DS2
    # drug_fea = pd.read_csv("NDD/DS2/simMatrix.csv", index_col=False, header=None, dtype=np.float64).to_numpy()
    # interaction = pd.read_csv("NDD/DS2/ddiMatrix.csv",index_col=False, header=None, dtype=np.int32).to_numpy()

    train = []
    label = []
    tmp_fea=[]
    drug_fea_tmp = []
    for i in range(0, interaction.shape[0]):
        for j in range(0, interaction.shape[1]):
            if i in candidates and j in candidates:     # added by sbn
                label.append(interaction[i,j])
                drug_fea_tmp = list(drug_fea[i])
                if seperate:

                     tmp_fea = (drug_fea_tmp,drug_fea_tmp)

                else:
                     tmp_fea = drug_fea_tmp + drug_fea_tmp
                train.append(tmp_fea)
    return np.array(train), label

In [ ]:
# Helper function for computing performance of Neural network
def calculate_performace(test_num, pred_y,  labels):
    tp =0
    fp = 0
    tn = 0
    fn = 0
    for index in range(test_num):
        if labels[index] ==1:
            if labels[index] == pred_y[index]:
                tp = tp +1
            else:
                fn = fn + 1
        else:
            if labels[index] == pred_y[index]:
                tn = tn +1
            else:
                fp = fp + 1 
    acc = float(tp + tn)/test_num
    if tp == 0 and fp == 0:
        precision = 0
        MCC = 0
        sensitivity = float(tp)/ (tp+fn)
        specificity = float(tn)/(tn + fp)
    else:
        precision = float(tp)/(tp+ fp)
        sensitivity = float(tp)/ (tp+fn)
        specificity = float(tn)/(tn + fp)
        MCC = float(tp*tn-fp*fn)/(np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)))
    return acc, precision, sensitivity, specificity, MCC

In [ ]:
# Changes the format of input data to Neural network
def transfer_array_format(data):
    formated_matrix1 = []
    formated_matrix2 = []
    for val in data:
        formated_matrix1.append(val[0])
        formated_matrix2.append(val[1])
    return np.array(formated_matrix1), np.array(formated_matrix2)

# Preprocess the labels
def preprocess_labels(labels, encoder=None, categorical=True):
    if not encoder:
        encoder = LabelEncoder()
        encoder.fit(labels)
        y = encoder.transform(labels).astype(np.int32)
    if categorical:
        y = np_utils.to_categorical(y)
    return y, encoder
# Preprocess the names
def preprocess_names(labels, encoder=None, categorical=True):
    if not encoder:
        encoder = LabelEncoder()
        encoder.fit(labels)
    if categorical:
        labels = np_utils.to_categorical(labels)
    return labels, encoder

In [ ]:
# Generate the model object
def NDD(input_dim): 
    # model = Sequential()
    # model.add(Dense(input_dim=input_dim, output_dim=400,init='glorot_normal'))
    # model.add(Activation('relu'))
    # model.add(Dropout(0.5))
    # model.add(Dense(input_dim=400, output_dim=300,init='glorot_normal'))
    # model.add(Activation('relu'))
    # model.add(Dropout(0.5))
    # model.add(Dense(input_dim=300, output_dim=2,init='glorot_normal'))
    # model.add(Activation('sigmoid'))
    # sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    # model.compile(loss='binary_crossentropy', optimizer=sgd)
    # return model

    model = Sequential([
        Dense(400, input_dim=input_dim, kernel_initializer='glorot_normal'),
        Activation('relu'),
        Dropout(0.5),
        Dense(300, kernel_initializer='glorot_normal'),
        Activation('relu'),
        Dropout(0.5),
        Dense(2, kernel_initializer='glorot_normal'),
        Activation('sigmoid')
    ])
    # sgd = optimizers.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    sgd = optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy', optimizer=sgd)
    return model

In [ ]:
# Main function for executing Neural network
def DeepMDA(candidates=None):
    X, labels = prepare_data(candidates, seperate = True)

    X_data1, X_data2 = transfer_array_format(X) 
    X=0
    y, encoder = preprocess_labels(labels)# labels labels_new
    X= np.concatenate((X_data1, X_data2), axis = 1)
    num = np.arange(len(y))
    np.random.shuffle(num)
    X_data1 = X_data1[num]
    X_data2 = X_data2[num]
    y = y[num]
    num_cross_val = 5
    all_performance_DNN = []
    for fold in range(num_cross_val):
        train_label = np.array([x for i, x in enumerate(y) if i % num_cross_val != fold])
        test_label = np.array([x for i, x in enumerate(y) if i % num_cross_val == fold])
        train1 = np.array([x for i, x in enumerate(X_data1) if i % num_cross_val != fold])
        test1 = np.array([x for i, x in enumerate(X_data1) if i % num_cross_val == fold])
        train2 = np.array([x for i, x in enumerate(X_data2) if i % num_cross_val != fold])
        test2 = np.array([x for i, x in enumerate(X_data2) if i % num_cross_val == fold])
     
        zerotest=0
        nozerotest=0
        zerotrain=0
        nozerotrain=0
        real_labels = []
        for val in test_label:
            if val[0] == 1:
                nozerotest=nozerotest+1
                real_labels.append(1)
            else:
                zerotest=zerotest+1
                real_labels.append(0)
        train_label_new = []
        for val in train_label:
            if val[0] == 1:
                zerotrain=zerotrain+1
                train_label_new.append(1)
            else:
                nozerotrain=nozerotrain+1
                train_label_new.append(0)
       
        prefilter_train = np.concatenate((train1, train2), axis = 1)
        prefilter_test = np.concatenate((test1, test2), axis = 1)
        
        model_DNN = NDD(prefilter_train.shape[1])
        train_label_new_forDNN = np.array([[0,1] if i == 1 else [1,0] for i in train_label_new])

        model_DNN.fit(prefilter_train,train_label_new_forDNN,batch_size=100,epochs=20,shuffle=True,validation_split=0)
        # proba = model_DNN.predict_classes(prefilter_test,batch_size=200,verbose=True)
        proba = np.argmax(model_DNN.predict(prefilter_test, batch_size=200, verbose=1), axis=1)
        # ae_y_pred_prob = model_DNN.predict_proba(prefilter_test,batch_size=200,verbose=True)
        ae_y_pred_prob = model_DNN.predict(prefilter_test,batch_size=200,verbose=True)
        acc, precision, sensitivity, specificity, MCC = calculate_performace(len(real_labels), proba,  real_labels)
        fpr, tpr, auc_thresholds = roc_curve(real_labels, ae_y_pred_prob[:,1])
        auc_score = auc(fpr, tpr)
        precision1, recall, pr_threshods = precision_recall_curve(real_labels, ae_y_pred_prob[:,1])
        aupr_score = auc(recall, precision1)
        all_F_measure=np.zeros(len(pr_threshods))
        for k in range(0,len(pr_threshods)):

           if (precision1[k]+precision1[k])>0:
              all_F_measure[k]=2*precision1[k]*recall[k]/(precision1[k]+recall[k])
           else:
              all_F_measure[k]=0

        max_index=all_F_measure.argmax()
        predicted_score=np.zeros(len(real_labels))
        threshold=pr_threshods[max_index]
        p=ae_y_pred_prob[:,1]
        predicted_score[p>threshold]=1
        f=f1_score(real_labels,predicted_score)
        recall=recall_score(real_labels, predicted_score)
        precision1=precision_score(real_labels, predicted_score)
        print("RAW DNN:",recall, precision1,'auc:', auc_score,'aupr', aupr_score,f)
        all_performance_DNN.append([recall,precision1,auc_score,aupr_score,f])
    print('recall,precision,auc_score,aupr_score,fscore')
    print(np.mean(np.array(all_performance_DNN), axis=0))

In [ ]:
path_list = [
    # "NDD/DS1/chem_Jacarrd_sim.csv",
    # "NDD/DS1/drug_drug_matrix.csv",
    "NDD/DS1/enzyme_Jacarrd_sim.csv",
    # "NDD/DS1/indication_Jacarrd_sim.csv",
    "NDD/DS1/offsideeffect_Jacarrd_sim.csv",
    "NDD/DS1/pathway_Jacarrd_sim.csv",
    "NDD/DS1/sideeffect_Jacarrd_sim.csv",
    # "NDD/DS1/target_Jacarrd_sim.csv",
    # "NDD/DS1/transporter_Jacarrd_sim.csv",
    "NDD/DS1/integratedDS1.txt",
    "NDD/DS2/simMatrix.csv"
]

# Main cell
similarity_matrix = read_similarity_matrix(path_list[-2])
mds = MDS(n_components=2, dissimilarity='precomputed')
embeddings = mds.fit_transform(similarity_matrix)
labels, kmeans = make_groups(embeddings, 300)
candidates = find_candidate(embeddings, kmeans)
DeepMDA(candidates)